# Transcript objects

The below code takes a GFF and goes through it line by line to extract the genetic features (genes)

My code focusses on transcripts rather than genes because transcripts should have unique coding sequences whereas genes can have multiple transcripts depending on alternative splicing etc.

The GFF parser is not the best one around so it is the most likely place to have my code break.
You will therefore need to make sure your GFF conforms to a few rules if you want it to work. Alternatively you can modify the `GFF_line` object and `hash_gff` function above.

Rules:
 - Every transcript must have a unique ID which is specified using the variable index_label
 - All features of that transcript should share that unique ID (ie CDS, intron, exon, 5'-UTR or 3'-UTR should have this label in their attributes 
 - transcripts should be identified as feature type `mRNA` *case sensitive*
 - The coding sequences should be feature type `CDS`
 - Exons are the sum of CDS + UTRs ie exons and CDS are not the same thing

e.g.

    C0077	AUGUSTUS	gene	8157	13385	0.05	-	.	gene_id=g1
    C0077	AUGUSTUS	mRNA	8157	13385	0.05	-	.	gene_id=g1;transcript_id=g1.t1
    C0077	AUGUSTUS	exon	8157	8487	0.59	-	.	gene_id=g1;transcript_id=g1.t1
    C0077	AUGUSTUS	3'-UTR	8157	8340	0.59	-	.	gene_id=g1;transcript_id=g1.t1
    C0077	AUGUSTUS	CDS	8341	8487	1	-	0	gene_id=g1;transcript_id=g1.t1
    C0077	AUGUSTUS	exon	8733	8837	1	-	.	gene_id=g1;transcript_id=g1.t1
    C0077	AUGUSTUS	CDS	8733	8837	1	-	0	gene_id=g1;transcript_id=g1.t1
    C0077	AUGUSTUS	exon	9139	9201	1	-	.	gene_id=g1;transcript_id=g1.t1
    C0077	AUGUSTUS	CDS	9139	9201	1	-	0	gene_id=g1;transcript_id=g1.t1
    C0077	AUGUSTUS	exon	9875	9951	0.95	-	.	gene_id=g1;transcript_id=g1.t1
    C0077	AUGUSTUS	CDS	9875	9951	0.95	-	2	gene_id=g1;transcript_id=g1.t1
    C0077	AUGUSTUS	exon	10390	10471	1	-	.	gene_id=g1;transcript_id=g1.t1
    C0077	AUGUSTUS	CDS	10390	10471	1	-	0	gene_id=g1;transcript_id=g1.t1
    C0077	AUGUSTUS	exon	11106	11154	0.94	-	.	gene_id=g1;transcript_id=g1.t1
    C0077	AUGUSTUS	CDS	11106	11154	0.94	-	1	gene_id=g1;transcript_id=g1.t1
    C0077	AUGUSTUS	exon	12646	13385	0.69	-	.	gene_id=g1;transcript_id=g1.t1
    C0077	AUGUSTUS	CDS	12646	13259	0.69	-	0	gene_id=g1;transcript_id=g1.t1
    C0077	AUGUSTUS	5'-UTR	13260	13385	0.31	-	.	gene_id=g1;transcript_id=g1.t1

** warning **    
The delimiters used to split the attribute field and the attribute field keys from their values are variable across formats
You need to specify which ones are used. e.g.

    gene_id=g1;transcript_id=g1.t1

The two fields (`gene_id` and `transcript_id`) are split by `;` which I call `info_delimiter` and each field is then split into its key (e.g. `transcript_id`) and value (e.g. `g1.t1`) by  the info_field_delimiter "=".

You can specify the delimiters in your file when you run my functions.

Some GFF files use different delimiters and even worse, some use just spaces - this is the most likely place for the code to break - so be careful.




---
# Read the transcripts into a dictionary of Transcript objects

Here we use the hash_gff function to read all the transcripts into a dictionary of transcript objects
Each transcript is keyed in the dictionary by its ID

We also read the reference genome into a dictionary.

*Depending on the number of transcripts this could take a couple minutes*

In [1]:
from annotation import Transcript
from Bio import SeqIO

GFF_file = "/scratch/research/references/chlamydomonas/5.3_chlamy_w_organelles_mt_minus/annotation/concatenated_GFF/final.strict.GFF3"
ref_genome_fasta = '/scratch/research/references/chlamydomonas/5.3_chlamy_w_organelles_mt_minus/chlamy.5.3.w_organelles_mtMinus.fasta'

transcript_dictionary = Transcript.hash_gff(GFF_file, index_label = 'ness_ID', info_delimiter=";", info_field_delimiter = '=', quiet=True)
ref_genome_dictionary = SeqIO.to_dict(SeqIO.parse(open(ref_genome_fasta), 'fasta'))

---
# Transcript objects
---

Transcript objects are my own custom class that keeps rich information about the features of a transcript that are encoded in the GFF.

You can access the information using the attributes and methods of each transcript

In [2]:
import random



example_ID = list(transcript_dictionary.keys())[random.randint(0, len(transcript_dictionary))]
example_transcript = transcript_dictionary[example_ID]

print("Transcript Name: ", example_transcript.name)
print("Transcript chromosomes: ", example_transcript.seqid)
print("Transcript start position: ", example_transcript.start)
print("Transcript end position: ", example_transcript.end)
print("Transcript DNA strand: ", example_transcript.strand)

Transcript Name:  26902562
Transcript chromosomes:  chromosome_3
Transcript start position:  5730310
Transcript end position:  5737375
Transcript DNA strand:  -


# Features
One of the most important attributes of the transcript is its `feats` which is a dictionary of all the features of the transcript.
Each key in the dictionary refers to a feature type and is itself a dictionary where the key is the start position of each feature and it's value is the GFF line that defined it. 

This effectively organizes all the features of the transcript and allows you to access all the information in the GFF


In [3]:
# All feature types
print("All the feature types:", list(example_transcript.feats.keys()))
# Look at CDS features
for k,v in example_transcript.feats['CDS'].items():
    print("Key is start", k)
    print("\tValue is GFF line", type(v))
    print ("\tValue has GFF info", v.seqid, v.start, v.end)

All the feature types: ['CDS', 'five_prime_UTR', 'exon', 'three_prime_UTR', 'mRNA', 'gene']
Key is start 5731814
	Value is GFF line <class 'annotation.GFF_line.GFF_line'>
	Value has GFF info chromosome_3 5731814 5732351
Key is start 5735126
	Value is GFF line <class 'annotation.GFF_line.GFF_line'>
	Value has GFF info chromosome_3 5735126 5735591
Key is start 5734775
	Value is GFF line <class 'annotation.GFF_line.GFF_line'>
	Value has GFF info chromosome_3 5734775 5734911
Key is start 5730953
	Value is GFF line <class 'annotation.GFF_line.GFF_line'>
	Value has GFF info chromosome_3 5730953 5731617
Key is start 5737039
	Value is GFF line <class 'annotation.GFF_line.GFF_line'>
	Value has GFF info chromosome_3 5737039 5737188
Key is start 5732599
	Value is GFF line <class 'annotation.GFF_line.GFF_line'>
	Value has GFF info chromosome_3 5732599 5734569
Key is start 5736796
	Value is GFF line <class 'annotation.GFF_line.GFF_line'>
	Value has GFF info chromosome_3 5736796 5736949
Key is start

# Accessing coding sequences and translations
There are also numerous methods of a transcript object that relate to its coding sequence, translation, codons, degeneracy etc.

 - To access actual sequence you need to provide the dictionary with the reference chromosomes as BioPython Seq Record
 - In general I return these sequences as lists (which is a bit annoying but it makes it easier to associate single positions with codons)
 - It is easy enough to use the string `.join(list)` method to bring it together


In [4]:
print('coding sequence:', "".join(example_transcript.cds(ref_genome_dictionary)))

coding sequence: ATGGTGGAAGTGGAGGTGCTGCCGCCGTCCGTGCCGGCGGTGATCTGCTTCGTGCTCGCCCTGACGGTCGGGCCGCGGGTCGCGGCTGTGACGGTCTTCGCCATCCTCGCCGCTACATTTGCCTTCAGCAGCGCCGGGGCGGGCATCCAGGCGCAGCTGGCGGCGGCGGCCGCACTGACGGCGGCGGTCCCCGCCACGTTCCTGTGGCTGCAGCGGCGGCGAGCGCCGCCACAGCAGCGGCAGCAGCAGCAGCCCGCCGCGCTACTGCCCCCAGAGCAAGAGCAACTCGCTGGCCCCAGACCAGGAGGCCCAGCCGCCACGCCCGCGCCGGCGGCAGTAGCAGGCAGTAGCAGCAGCCTTGCCGCTGCGCTACTGCCGCAGCTAGGCTTGCGCATGCAGCGCACCGCTGCGTGGGCTGCTGCGACGTCGGCGGACCTGGAGGAAGGGCTCAGCCTCCAGCGGCCATCGCCGGCACCGCAGGCGTCTGCCGCTCCCTCCAGCCCCTCCGACACCTCAGCAGCGCCTCCCCCAGGCAGCGCGGCAGGCGCGGCGGACGCAGCAGCCGCCAACCACCTCTCCCGTCTGCTCCCGGATTTCAGTGTGGGAGAGGGCGGCGGGCAGGAACAGCCGCTCCAGCTGTGGCGCGGCGGGCGGCAGCAGCCGGGTGCCTCCGCAGCGCTGACGGAGGTGGTGGGCGTCACGGGCGGCGGCAACACCGCCAGCGGCAACGCCACCCGCTGCACCACCACCTGGTCATCACTGCCGCTGTTCTGCTTCGCAGAGGTGCTGAGCTGGATGGAGGACGACTGCGACCGGCGGCGGCTGCGGCTGGTGTGCCGGGACTGGCGCGCCGCCGCCGCCGCCGCCACCACGCACCTGGCGCCGCCGCGGTCGGGTGCCCGACACCTGGGTGCGCTGGCGGCGGCGTTCCCCGGCCTCACAGCCCTGGACCTGTCAGCTTGCCTGGCGCACCTGACGCCGCC

In [5]:
print('translation sequence:', "".join(example_transcript.aa(ref_genome_dictionary)))

translation sequence: MVEVEVLPPSVPAVICFVLALTVGPRVAAVTVFAILAATFAFSSAGAGIQAQLAAAAALTAAVPATFLWLQRRRAPPQQRQQQQPAALLPPEQEQLAGPRPGGPAATPAPAAVAGSSSSLAAALLPQLGLRMQRTAAWAAATSADLEEGLSLQRPSPAPQASAAPSSPSDTSAAPPPGSAAGAADAAAANHLSRLLPDFSVGEGGGQEQPLQLWRGGRQQPGASAALTEVVGVTGGGNTASGNATRCTTTWSSLPLFCFAEVLSWMEDDCDRRRLRLVCRDWRAAAAAATTHLAPPRSGARHLGALAAAFPGLTALDLSACLAHLTPPRMRHLSPLRHLSRLTLGCPSPASAAATEDLLRAQLQQQQQQQQPHNPYLADAAAAAAAPGLLAADSWRRQRCTTSTSESSGGAAAVTDAMVQELVAALLRGGGGGGRRQAGAAGAGGGGGGGGGAGGGGGGGRGELRVLSIAQCVRVTDAGVLALTALTGLTSLDLCGCCGVSDVGVMLLARLPLLQGLQLAWCVKVSNAGLRGLAVLPRLSHLSVAGCPLVSEAGVAGLSTLSRLACLDLTHLGITQQRATVTDAALSALCGGGTPGTAGGGGGGGAASAARDQGGGAPLAPPPHPRLTRLAFGGGRGGGTRVTDAGLAGLAAAQPQLAGLTLLSLQFVSDAGLEAVVAALGDLTSLCVRGCGPGVTRGCVAALTEAVAARGGRPAAPLPPRAEPWWRLRAAQAAAAAAGYRVGGAGGLAMPRLQELSLMHNPFVALRDEDVARLCGAAAATAAAAAAAGGGGGGCLTSLSLGGAVLLVPQLPAPAPPPPQQPPQPQPPQQQQQPQQQQQQQQQQQPQQQQPGAAEVVGGHDALAAGGDGGAHGPDGGYHNIAAAELAAAAVANAPGTAPQGVLLALGAGPGGGGGGGGAGGGGGGGGGGGGAGAAGGGGGGGGGGGGGGGGAAAAAVNEPPDALRSVGLGLVSLRAMA

### For every position in the coding sequence you can also get its degeneracy or position in the reference:

    

In [6]:
cds = example_transcript.cds(ref_genome_dictionary)
degeneracy = example_transcript.cds_degen(ref_genome_dictionary)
cds_map = example_transcript.cds_map()
for position in range(12): # lets only look at the first 12 positions to avoid filling the screen!
    print("base:", cds[position],"\tdegeneracy:",  degeneracy[position], "\tgenomic position:",  cds_map[position])

base: A 	degeneracy: 0 	genomic position: 5737188
base: T 	degeneracy: 0 	genomic position: 5737187
base: G 	degeneracy: 0 	genomic position: 5737186
base: G 	degeneracy: 0 	genomic position: 5737185
base: T 	degeneracy: 0 	genomic position: 5737184
base: G 	degeneracy: 4 	genomic position: 5737183
base: G 	degeneracy: 0 	genomic position: 5737182
base: A 	degeneracy: 0 	genomic position: 5737181
base: A 	degeneracy: 2 	genomic position: 5737180
base: G 	degeneracy: 0 	genomic position: 5737179
base: T 	degeneracy: 0 	genomic position: 5737178
base: G 	degeneracy: 4 	genomic position: 5737177


---
# Introns
---

Although introns are not always recorded in GFFs my transcript class has a `introns` method that will infer the introns and add them to the `feats` dictionary as GFF line objects

You can choose to infer introns from the exon or CDS features using the argument `which_exons` that specifies the feature type to use 
       - but be cautious as there are introns within UTR and using CDS may miss introns.


In [11]:
example_transcript.introns(which_exons='exon')

for key, intron in example_transcript.feats['intron'].items():
    print(intron.start, intron.end)

5732352 5732598
5734912 5735125
5731618 5731813
5736950 5737038
5735592 5735764
5734570 5734774
5736430 5736795
